In [1]:
import os
import jaxtyping
from pathlib import Path
import json

import time
from dataclasses import dataclass
from pathlib import Path
from typing import Callable

import einops
import numpy as np
import torch as t
import torch.nn as nn
import torchrl
import wandb
import tqdm
import tabulate

from eindex import eindex
from jaxtyping import Float, Int
from rich import print as rprint
from rich.table import Table
from tabulate import tabulate
from torch import Tensor
from transformer_lens import HookedTransformer, utils
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torchrl.envs.llm import ChatEnv
from torchrl.modules.llm import TransformersWrapper
from tqdm import tqdm

device = t.device("cuda" if t.cuda.is_available() else "mps" if t.backends.mps.is_available() else "cpu")
print(f"Device: {device}")

Device: mps


In [ ]:
# load artifact from wandb
run = wandb.init()
artifact = run.use_artifact('djdumpling-yale/rlhf_transformers/model-gpt2_20250730-004324:v0', type='model')
model_dir = artifact.download()

# set location locally 
model_path = os.path.join(model_dir, "final_model.pt")
state_dict = t.load(model_path, map_location=device)

# get weights from RLHFed model
base_state_dict = {}
for k, v in state_dict.items():
    if k.startswith('base_model.'):
        new_key = k.replace('base_model.', '')
        base_state_dict[new_key] = v

# load into gpt architecture
base_model = HookedTransformer.from_pretrained("gpt2", device=device)
base_model.load_state_dict(base_state_dict)
base_model.eval()

# if TRL comes in later?
model = TransformersWrapper(model=base_model, tokenizer=base_model.tokenizer,input_mode="text")

wandb: Currently logged in as: djdumpling (djdumpling-yale) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-gpt2_20250730-004324:v0, 643.08MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:6.9 (93.6MB/s)


Model directory: /Users/alexwa/Documents/GitHub/rl/rlhf_transformer/artifacts/model-gpt2_20250730-004324:v0
Loading model from: /Users/alexwa/Documents/GitHub/rl/rlhf_transformer/artifacts/model-gpt2_20250730-004324:v0/final_model.pt
Loaded pretrained model gpt2 into HookedTransformer
Successfully loaded model with TorchRL wrapper!


In [5]:
os.environ["WANDB_NOTEBOOK_NAME"] = "disabled"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

base_model.generate("This movie was really", max_new_tokens=10, temperature=0.0)

  0%|          | 0/10 [00:00<?, ?it/s]

'This movie was really great. The music was great, the characters were'